# Built-in Functions

| ANSI SQL Standard | POstgreSQL Extensions |
--------- | -------
sum  | trunc
max | cbrt
min | pi
cast | random
count | to_num
floor | to_char
ceiling | now


## Business Use Cases

+ Encaptsulate business logic
+ Enforce semantic relationships
+ Validate and transoform data
+ control access to sensitive date
+ Encourage code reuse
+ Don't Repeat YOurself(DRY)




+ Setting up a working environment
+ Creating your first function
+ Data types
+ Polymorphic functions
+ Function overloading
+ Stored procedures
+ Error handling

In [1]:
select CURRENT_TIMESTAMP

(1 row(s) affected)

Total execution time: 00:00:00.002

current_timestamp
2024-09-07 23:42:02.844127+08


In [2]:
select * from customers limit 1

(1 row(s) affected)

Total execution time: 00:00:00.002

customerid,firstname,lastname,address1,address2,city,state,zip,country,region,email,phone,creditcardtype,creditcard,creditcardexpiration,username,password,age,income,gender
1,RNPOJG,FQIVBKWIZC,1287944454 Dell Way,,XCXNWIS,ND,42572,US,1,FQIVBKWIZC@dell.com,1287944454,5,1890860486775636,2012/06,user1,password,49,80000,M


In [4]:
SELECT
    c.firstname,
    c.lastname,
    o.orderid,
    o.netamount
    from customers C
    join orders O
    on c.customerid = o.customerid limit 10;

(10 row(s) affected)

Total execution time: 00:00:00.002

firstname,lastname,orderid,netamount
RKLQYY,HPUIUYBWZX,10677,5.08
GWPSHI,QWRLBUPCHY,2337,39.06
FCHDNP,FQZNEVVHNA,9077,323.30
HOMUAY,BCTJTZIEZQ,6239,341.44
UZCYBH,NKKNSRDQGC,1187,285.39
YADYNS,VBVFOAENRH,3710,350.87
ZGYPBI,NKKQMREHPQ,379,227.45
ZGYPBI,NKKQMREHPQ,9447,83.31
PFIWAF,MQGNVHRTNH,3075,33.63
PQJVZM,PCCNFVKLZE,5019,256.30


In [5]:
create or replace view cust_orders 
as 
SELECT
    c.firstname,
    c.lastname,
    o.orderid,
    o.netamount
    from customers C
    join orders O
    on c.customerid = o.customerid limit 10;

CREATE VIEW

Total execution time: 00:00:00.003

In [18]:
select * from cust_orders;

: relation "cust_orders" does not exist

Total execution time: 00:00:00

In [7]:
create function cust_orders()
    returns table (firstname varchar, lastname varchar, orderid int, netamount numeric)
    as $$
SELECT
    c.firstname,
    c.lastname,
    o.orderid,
    o.netamount
    from customers C
    join orders O
    on c.customerid = o.customerid limit 10
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [8]:
select * from cust_orders();

(10 row(s) affected)

Total execution time: 00:00:00.001

firstname,lastname,orderid,netamount
RKLQYY,HPUIUYBWZX,10677,5.08
GWPSHI,QWRLBUPCHY,2337,39.06
FCHDNP,FQZNEVVHNA,9077,323.30
HOMUAY,BCTJTZIEZQ,6239,341.44
UZCYBH,NKKNSRDQGC,1187,285.39
YADYNS,VBVFOAENRH,3710,350.87
ZGYPBI,NKKQMREHPQ,379,227.45
ZGYPBI,NKKQMREHPQ,9447,83.31
PFIWAF,MQGNVHRTNH,3075,33.63
PQJVZM,PCCNFVKLZE,5019,256.30


In [11]:
create or replace function cust_orders(customerid int)
    returns table (firstname varchar, lastname varchar, orderid int, netamount numeric)
    as $$
SELECT
    c.firstname,
    c.lastname,
    o.orderid,
    o.netamount
    from customers C
    join orders O
    on c.customerid = o.customerid 
    where c.customerid = cust_orders.customerid
    limit 10
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [17]:
select * from cust_orders(42);

(1 row(s) affected)

Total execution time: 00:00:00.001

firstname,lastname,orderid,netamount
IGAITZ,ADIDJMAYUZ,11465,202.29


In [19]:
drop view if exists cust_orders;
drop function if exists cust_orders();
drop function if exists cust_orders(int);

NOTICE: view "cust_orders" does not exist, skipping

DROP VIEW

DROP FUNCTION

DROP FUNCTION

Total execution time: 00:00:00.002

## Input Parameters and Return Values

### Multiple input parameters


In [21]:
create or replace function my_pow(x double precision, y double precision)
returns double precision
as $$ 
select power(x, y)
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [23]:
select my_pow(1,2)

(1 row(s) affected)

Total execution time: 00:00:00.002

my_pow
1


In [25]:
select my_pow(PI(), log(42))

(1 row(s) affected)

Total execution time: 00:00:00.001

my_pow
6.412068866243689


### Default values

In [27]:
create or replace function my_default(x int = 42) 
RETURNS INT
as $$
    SELECT x;
    $$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [28]:
select my_default()

(1 row(s) affected)

Total execution time: 00:00:00.003

my_default
42


### Using arrays for multiple input values

In [32]:
create or replace function array_sum(int_array int[])
RETURNs int 
as $$
    select sum(el) 
    from unnest(int_array) as arr(el);
    $$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [34]:
select array_sum(array[1,2,3])

(1 row(s) affected)

Total execution time: 00:00:00.001

array_sum
6


## Arrays and multiple return values


In [37]:
create or replace function array_sum_avg(int_array int[])
RETURNS table (array_sum bigint, array_avg numeric)
as $$
select sum(el), avg(el)::numeric(5,2)
from unnest(int_array) as arr(el);
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [39]:
select array_sum_avg(array[1,2,3]) as "Record type";



(1 row(s) affected)

Total execution time: 00:00:00.001

Record type
"(6,2.00)"


In [41]:
select * from array_sum_avg(array[1,2,3])

(1 row(s) affected)

Total execution time: 00:00:00.001

array_sum,array_avg
6,2.00


## Output arguments


In [3]:
create or replace function get_cust_name (
    in id int,
    out firstname varchar,
    out lastname varchar
) as $$
    select c.firstname, c.lastname from customers C
    where c.customerid = id;
    $$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [6]:
SELECT * from get_cust_name(42);

(1 row(s) affected)

Total execution time: 00:00:00.007

firstname,lastname
IGAITZ,ADIDJMAYUZ


## Returns a table revisited

In [7]:
create or replace function get_cust_names(id1 int, id2 int) 
returns table (firstname varchar, lastname varchar)
as $$
select c.firstname, c.lastname from customers C
where c.customerid between id1 and id2;
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

In [10]:
select * from get_cust_names(1, 5);

(5 row(s) affected)

Total execution time: 00:00:00.003

firstname,lastname
RNPOJG,FQIVBKWIZC
RKLQYY,HPUIUYBWZX
GWPSHI,QWRLBUPCHY
ONCPHI,TZIJOMZQJJ
CIDXWX,GNYGKZXSCR


In [14]:
-- drop function if exists get_cust_names(id1 int, id2 int)

create or replace function get_cust_names2(
    inout id1 int, id2 int,
    out firstname varchar,
    out lastname varchar
) as $$
select c.customerid, c.firstname, c.lastname from customers c
where c.customerid between id1 and id2
-- order by c.customerid desc
$$ language sql

CREATE FUNCTION

Total execution time: 00:00:00.001

In [16]:
select 
id1 as customerid,
firstname, 
lastname
 from get_cust_names2(1,5);

(1 row(s) affected)

Total execution time: 00:00:00.001

customerid,firstname,lastname
1,RNPOJG,FQIVBKWIZC


In [17]:

select 
id1 as customerid,
firstname, 
lastname
 from get_cust_names2(-1,-25);

(1 row(s) affected)

Total execution time: 00:00:00.001

customerid,firstname,lastname
NULL,NULL,NULL


In [19]:
select 
firstname, 
lastname
 from get_cust_names(-1,-25);

(0 row(s) affected)

Total execution time: 00:00:00.001

firstname,lastname


In [ ]:
drop function if EXISTS get_cust_name(id int, out firstname varchar, out lastname varchar);
drop function if EXISTS get_cust_names(id1 int, id1 int, out firstname varchar, out lastname varchar);
drop function if EXISTS get_cust_names(input id1 int, id1 int, out firstname varchar, out lastname varchar);
drop function if EXISTS get_cust_names(id1 int, id1 int);

## Calling Functions

In [20]:
create or replace function call_me(x int, y int, sw boolean = True) returns INT
as $$
select x + y where sw 
union ALL
select x - y where not sw;
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.003

In [21]:
select call_me(42, -42 ) as "Positional arguments",
call_me(x := 42, y := -42) as "Named arguments",
call_me(42, -42, False) as "Positional arguments with switch",
call_me(42, -42, sw := False) as "Mixed positional and named";

(1 row(s) affected)

Total execution time: 00:00:00.002

Positional arguments,Named arguments,Positional arguments with switch,Mixed positional and named
0,0,84,84


In [23]:
select call_me(x := 42, y := -42, False) as "Named Followed by positional";

: positional argument cannot follow named argument

Total execution time: 00:00:00

In [2]:
create or replace function call_me2(x int, y int, sw boolean = false) returns INT
as $$
select x + y where sw
union ALL
select x - y where not sw
$$ language sql;

CREATE FUNCTION

执行时间总计: 00:00:00

In [3]:
select call_me2(42, -42 ) as "Positional arguments",
call_me2(x := 42, y := -42) as "Named arguments",
call_me2(42, -42, False) as "Positional arguments with switch",
call_me2(42, -42, sw := False) as "Mixed positional and named";

(1 row(s) affected)

执行时间总计: 00:00:00.024

Positional arguments,Named arguments,Positional arguments with switch,Mixed positional and named
84,84,84,84


In [4]:
SELECT
    c.column_name,
    pgd.description
FROM
    pg_catalog.pg_statio_all_tables as st
    INNER JOIN pg_catalog.pg_description pgd ON (pgd.objoid = st.relid)
    INNER JOIN information_schema.columns c ON (pgd.objsubid = c.ordinal_position
        AND c.table_schema = st.schemaname AND c.table_name = st.relname)

(11 row(s) affected)

执行时间总计: 00:00:00.010

column_name,description
jobagentid,Agent that currently executes this job.
jstkind,"Kind of jobstep: s=sql, b=batch"
jstonerror,"What to do if step returns an error: f=fail the job, s=mark step as succeeded and continue, i=mark as fail but ignore it and proceed"
jlgstatus,"Status of job: r=running, s=successfully finished, f=failed, i=no steps to execute, d=aborted"
jslstatus,"Status of job step: r=running, s=successfully finished, f=failed stopping job, i=ignored failure, d=aborted"
jslresult,Return code of job step
employee_id,员工ID
first_name,名字
last_name,姓氏
email,邮箱


## Creating Functions in PL/pgSQL

Using Pl/pgSQL

+ Process blocks of computation on the server
+ Includes control structures for iteration and conditional processing
+ Supports typed variables for complex calculations
+ Inherits all user-defined types, functions, and operators
+ Procecdural language for creating functions and triggers


## Structure of a PL/pgSQL program



### Returning a table using the SQL language

In [2]:
create or replace FUNCTION get_cust_names(id1 int, id2 int)
returns table (firstname varchar, lastname varchar) AS
$$
select c.firstname, c.lastname from customers as C
where c.customerid BETWEEN id1 and id2
$$ language sql;

CREATE FUNCTION

Total execution time: 00:00:00.001

### Returing a table using teh PL/pgSQL language


In [3]:
create or replace function get_cust_names(id1 int, id2 int)
returns table(firstname varchar, lastname varchar)
as $$
BEGIN
    return query
    select c.firstname, c.lastname from customers as c
    where c.customerid BETWEEN id1 and id2;
end
$$ language plpgsql;

CREATE FUNCTION

Total execution time: 00:00:00.005

In [4]:
SELECT * from get_cust_names(1,5);

(5 row(s) affected)

Total execution time: 00:00:00.004

firstname,lastname
RNPOJG,FQIVBKWIZC
RKLQYY,HPUIUYBWZX
GWPSHI,QWRLBUPCHY
ONCPHI,TZIJOMZQJJ
CIDXWX,GNYGKZXSCR


### PL/pgSQL program structure



```
[<<label>>]
[ declare
    declarations ]
begin
    statements
end [label];
```

In [5]:
do $$ begin null;end; $$

DO

Total execution time: 00:00:00.001

In [7]:
do language plpgsql $$
<<get_ans>>
DECLARE
    the_answer int := 42;
begin
    raise notice 'The answer is %.', get_ans.the_answer;
end get_ans;
$$

NOTICE: The answer is 42.

DO

Total execution time: 00:00:00

### Declaring and using variables


#### Delcaring variables

In [10]:
do LANGUAGE plpgsql $$
-- name [constant ] type [ collate collation_name ] [not null] [ (default | := | =) expression ];

DECLARE
    myint INTEGER not null = 0;
    the_answer NUMERIC(2) := 42;
    phi CONSTANT double precision DEFAULT ( 1 + |/ 5) / 2;
    cust_row customers%rowtype;
    cust_firstname customers.firstname%type;
    myrow record;
BEGIN
    myint := -1; --myint = -1
    select * from customers into cust_row limit 1;
    select cust_row.firstname into cust_firstname limit 1;
    select 42::int as the_answer into myrow;
    raise notice 'Last name: %; First name: %; How young? %', cust_row.lastname, cust_firstname, myrow.the_answer;
    end;
    $$

NOTICE: Last name: FQIVBKWIZC; First name: RNPOJG; How young? 42

DO

Total execution time: 00:00:00.003